In [1]:
import numpy as np

In [123]:
class Fundamentalist():
    def __init__(self, a=0, b=0, c=1):
        self.a=a
        self.b=b
        self.c=c
    
    def get_expectation(self, last_price):
        return self.a + self.b * last_price
    
class Naive():
    def __init__(self, a=0, b=1, c=0):
        self.a=a
        self.b=b
        self.c=c
    
    def get_expectation(self, last_price):
        return self.a + self.b * last_price
    
class Adaptive():
    def __init__(self, a=0, b=0, c=0.5, w=0.5):
        self.a=a
        self.b=b
        self.c=c
        
        self.last_expectation = 0
        
    def get_expectation(self, last_price):
        self.last_expectation = self.last_expectation + self.w*(last_price - self.last_expectation)
        return self.last_expectation


class CobWebModel():
    def __init__(self, A=0, d=3/4, s=1, start_price=0, beta=1/2):
        self.A = A
        self.d = d
        self.s = s
        self.beta = beta
        
        self.agents = []
        self.fractions = []
        self.expectations = []
        
        self.prices = [start_price]
        
    def step(self):
        self.expectations.append(self.get_expectations())
        self.prices.append(self.get_new_price())
        self.fractions = self.get_new_fractions()
        
    
    def add_agent(self,agent, fraction):
        self.agents.append(agent)
        self.fractions.append(fraction)
        
    def get_expectations(self):
        return [agent.get_expectation(self.prices[-1]) for agent in self.agents]
        
    def get_new_price(self):
        return -1*(np.sum([self.s*self.fractions[i]*expectation for i, expectation in enumerate(self.expectations[-1])]) - self.A)/self.d
    
    def get_supply(self, expected_price):
        return self.s*expected_price
    
    def get_profit(self, expected_price, agent):
        supply = self.get_supply(expected_price)
        return supply*self.prices[-1] - 0.5*supply**2-agent.c
    
    def get_new_fractions(self):
        profits = [self.get_profit(self.expectations[-1][i], agent) for i,agent in enumerate(self.agents)]
        Z = np.sum([np.exp(self.beta*profit) for profit in profits])
        
        
        return [np.exp(self.beta*profit)/Z for profit in profits]

In [140]:
cwmodel = CobWebModel(start_price = 10)

In [141]:
cwmodel.add_agent(Naive(), 0.5)
cwmodel.add_agent(Fundamentalist(), 0.5)

In [150]:
cwmodel.step()

In [151]:
cwmodel.fractions

[0.6224593312018546, 0.37754066879814546]